In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

import pandas as pd

In [2]:
qdrant_client = QdrantClient(
    url="http://localhost:6333"
)

qdrant_client.create_collection(
    collection_name="Amazon-items-collection-00",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE))

True

In [3]:
df_items = pd.read_json('../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl', lines=True)

### Concatenate title and featues

In [4]:
def preprocess_data(row):
    return f"{row['title']} {''.join(row['features'])}"

In [5]:
df_items["preprocessed_data"] = df_items.apply(preprocess_data, axis=1)

In [6]:
df_items.head(2)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author,preprocessed_data
0,Sports & Outdoors,Lowrance Hook2-4x Bullet Skimmer Ce Row One Size,4.2,314,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Lowrance,"[Electronics, Car & Vehicle Electronics, Marin...",{'Item Package Dimensions L x W x H': '10.63 x...,B0773K75DM,NaN,NaN,NaN,Lowrance Hook2-4x Bullet Skimmer Ce Row One Size
1,Cell Phones & Accessories,"Maxjoy Airpod Pro 2 Case 2022 Crystal Clear, A...",4.2,145,[【Perfect Compatibility】: This protective case...,[],10.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'AIRSPO Clear AirPods Pro 2 Case', ...",Maxjoy,"[Electronics, Headphones, Earbuds & Accessorie...",{'Package Dimensions': '3.58 x 3.15 x 1.3 inch...,B0BJ6986CY,NaN,NaN,NaN,"Maxjoy Airpod Pro 2 Case 2022 Crystal Clear, A..."


### Sample 50 items 

In [7]:
df_sample = df_items.sample(50, random_state=42)

### Embedding function

In [8]:
import openai
from dotenv import load_dotenv
import os

load_dotenv()
openai.api_key = os.getenv("OPEN_AI_KEY")

In [9]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=[text],
        model=model
    )
    return response.data[0].embedding

### Embed data

In [10]:
data_to_emded = df_sample["preprocessed_data"].tolist()
pointstructs = []
for i, data in enumerate(data_to_emded):
    embedding = get_embedding(data)
    pointstructs.append(
        PointStruct(
        id=i,
        vector=embedding,
        payload={"text": data}
    )
)

In [11]:
pointstructs

[PointStruct(id=0, vector=[0.019885430112481117, 0.016089024022221565, -0.002181858755648136, 0.011249410919845104, -0.05179461091756821, -0.07274475693702698, -0.019261658191680908, 0.007345456629991531, 0.010259978473186493, 0.009695357643067837, -0.02011127956211567, 0.027596546337008476, -0.04929951950907707, -0.021993350237607956, 0.008356397971510887, 0.026650134474039078, -0.023122593760490417, 0.007479890249669552, -0.0425240620970726, 0.005345083307474852, 0.014669403433799744, 0.06521646678447723, 0.03219955787062645, -0.00047723957686685026, -0.028220318257808685, -0.028220318257808685, 0.005716120358556509, 0.03183389827609062, 0.04101840406656265, -0.00325598381459713, 0.027682583779096603, -0.03349011763930321, 0.006436684634536505, -0.014067140407860279, -0.007200268097221851, 0.017809774726629257, 0.0037964072544127703, -0.0071088531985878944, 0.02396146021783352, -0.014292988926172256, 0.027101831510663033, 0.04334142059087753, 0.03400634601712227, -0.0572579950094223,

### Write embedded data to Qdrant

In [12]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-00",
    wait=True,
    points=pointstructs
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

### Function for data retrieval

In [13]:
def retrieve_data(query):
    query_embedding = get_embedding(query)
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=10
    )
    return results

In [14]:
try:
    qdrant_client.get_collections()
    print("Qdrant is up!")
except Exception as e:
    print(f"Connection failed: {e}")

Qdrant is up!


In [15]:
retrieve_data("What earphones can I get?").points

[ScoredPoint(id=11, version=0, score=0.5329315, payload={'text': 'Wireless Earbuds Bluetooth 5.3 Headphones with 4-Mics Clear Call and ENC Noise Cancelling, Bluetooth Earbuds Touch Control Stereo Sound with LED Display, Waterproof Running Headphones for Workout Enjoy Bluetooth 5.3 and Fast Auto Pairing: Latest Bluetooth 5.3 technology achieves lower latency and stronger anti-interference. MD058A true bluetooth headphone offer audio delays much lower than 65ms. Feature reduces the number of disconnections of the ear buds in complex environments(places with many people, such as subways and buses). Adopting hall switch, after the first connection, it will connect with your device as soon as the charging case opens.4-Mic and ENC Call Noise Reduction: Each headset has 2 built-in microphones, equipped with Environmental Noise Reduction (ENC) technology that can suppress 80% of the interfering background noise in the calling environment, intelligently enhances human voicethrough the four micr